# Editor Implementation Notebook
This notebook implements the features for editing based on the provided design document.

# Import Required Libraries
Import necessary libraries such as tkinter for GUI, PIL for image processing, and others as needed.

In [ ]:
# Import necessary libraries
import tkinter as tk
from tkinter import ttk, filedialog, colorchooser
from PIL import Image, ImageTk
import numpy as np
import json
import os
import sys
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple
import uuid
import copy

# Element Content Functionality
Implement tools for selecting characters from a library, adding and editing images, and enabling drag-and-drop interactions.

In [ ]:
class ElementManager:
    def __init__(self, canvas):
        self.canvas = canvas
        self.elements = {}
        self.selected_elements = set()
        self.current_element = None
        
    def add_text_element(self, text, x, y, font="Arial 12", color="#000000"):
        """Add a text element to the canvas"""
        element_id = str(uuid.uuid4())
        text_id = self.canvas.create_text(x, y, text=text, font=font, fill=color, tags=(element_id, "element", "text"))
        
        self.elements[element_id] = {
            "type": "text",
            "canvas_id": text_id,
            "text": text,
            "x": x,
            "y": y,
            "font": font,
            "color": color,
            "rotation": 0,
            "opacity": 1.0
        }
        return element_id
    
    def add_image_element(self, image_path, x, y, width=None, height=None):
        """Add an image element to the canvas"""
        element_id = str(uuid.uuid4())
        
        # Load image with PIL
        img = Image.open(image_path)
        if width and height:
            img = img.resize((width, height), Image.LANCZOS)
        photo = ImageTk.PhotoImage(img)
        
        # Keep a reference to prevent garbage collection
        if not hasattr(self, "photo_references"):
            self.photo_references = {}
        self.photo_references[element_id] = photo
        
        image_id = self.canvas.create_image(x, y, image=photo, anchor="nw", tags=(element_id, "element", "image"))
        
        self.elements[element_id] = {
            "type": "image",
            "canvas_id": image_id,
            "image_path": image_path,
            "x": x,
            "y": y,
            "width": width or img.width,
            "height": height or img.height,
            "rotation": 0,
            "opacity": 1.0
        }
        return element_id
    
    def select_element(self, element_id):
        """Select an element and show its selection handles"""
        if element_id in self.elements:
            self.selected_elements.add(element_id)
            self.highlight_element(element_id)
            self.current_element = element_id
            return True
        return False
    
    def deselect_all(self):
        """Deselect all elements"""
        for element_id in self.selected_elements:
            self.remove_highlight(element_id)
        self.selected_elements.clear()
        self.current_element = None
    
    def highlight_element(self, element_id):
        """Add selection handles around the element"""
        element = self.elements[element_id]
        canvas_id = element["canvas_id"]
        bbox = self.canvas.bbox(canvas_id)
        
        if not bbox:
            return
        
        # Create selection rectangle
        x1, y1, x2, y2 = bbox
        padding = 5
        selection_rect = self.canvas.create_rectangle(
            x1 - padding, y1 - padding, 
            x2 + padding, y2 + padding,
            outline="blue", width=2,
            tags=(f"{element_id}_selection", "selection")
        )
        
        # Create resize handles
        handle_size = 6
        handles = []
        
        # Eight handles (NW, N, NE, E, SE, S, SW, W)
        handle_positions = [
            (x1 - padding, y1 - padding, "nw-resize"),  # NW
            ((x1 + x2) / 2, y1 - padding, "n-resize"),  # N
            (x2 + padding, y1 - padding, "ne-resize"),  # NE
            (x2 + padding, (y1 + y2) / 2, "e-resize"),  # E
            (x2 + padding, y2 + padding, "se-resize"),  # SE
            ((x1 + x2) / 2, y2 + padding, "s-resize"),  # S
            (x1 - padding, y2 + padding, "sw-resize"),  # SW
            (x1 - padding, (y1 + y2) / 2, "w-resize"),  # W
        ]
        
        for x, y, cursor in handle_positions:
            handle = self.canvas.create_rectangle(
                x - handle_size/2, y - handle_size/2,
                x + handle_size/2, y + handle_size/2,
                fill="white", outline="blue",
                tags=(f"{element_id}_handle", "handle", cursor)
            )
            handles.append(handle)
        
        # Rotation handle
        rotation_handle = self.canvas.create_oval(
            (x1 + x2) / 2 - handle_size/2, y1 - padding - 20 - handle_size/2,
            (x1 + x2) / 2 + handle_size/2, y1 - padding - 20 + handle_size/2,
            fill="white", outline="blue",
            tags=(f"{element_id}_rotation", "rotation_handle")
        )
        
        # Rotation line
        rotation_line = self.canvas.create_line(
            (x1 + x2) / 2, y1 - padding,
            (x1 + x2) / 2, y1 - padding - 20,
            fill="blue",
            tags=(f"{element_id}_rotation_line", "rotation_line")
        )
        
        element["selection"] = {
            "rectangle": selection_rect,
            "handles": handles,
            "rotation_handle": rotation_handle,
            "rotation_line": rotation_line
        }
    
    def remove_highlight(self, element_id):
        """Remove selection handles for an element"""
        if element_id in self.elements and "selection" in self.elements[element_id]:
            selection = self.elements[element_id]["selection"]
            self.canvas.delete(selection["rectangle"])
            for handle in selection["handles"]:
                self.canvas.delete(handle)
            self.canvas.delete(selection["rotation_handle"])
            self.canvas.delete(selection["rotation_line"])
            del self.elements[element_id]["selection"]

# Transformation Operations
Define variables like _isDragging, _isResizing, and _isRotating, and implement complete transformation functionalities for position, size, rotation, and opacity adjustments.

In [ ]:
class TransformationManager:
    def __init__(self, canvas, element_manager):
        self.canvas = canvas
        self.element_manager = element_manager
        
        self._isDragging = False
        self._isResizing = False
        self._isRotating = False
        
        self.drag_start_x = 0
        self.drag_start_y = 0
        self.drag_element = None
        self.resize_handle = None
        self.resize_start_coords = (0, 0, 0, 0)  # x1, y1, x2, y2
        self.rotation_start_angle = 0
        
        self.setup_event_bindings()
    
    def setup_event_bindings(self):
        """Set up event bindings for transformations"""
        self.canvas.tag_bind("element", "<ButtonPress-1>", self.on_element_press)
        self.canvas.tag_bind("element", "<B1-Motion>", self.on_element_drag)
        self.canvas.tag_bind("element", "<ButtonRelease-1>", self.on_element_release)
        
        self.canvas.tag_bind("handle", "<ButtonPress-1>", self.on_resize_press)
        self.canvas.tag_bind("handle", "<B1-Motion>", self.on_resize_drag)
        self.canvas.tag_bind("handle", "<ButtonRelease-1>", self.on_resize_release)
        
        self.canvas.tag_bind("rotation_handle", "<ButtonPress-1>", self.on_rotation_press)
        self.canvas.tag_bind("rotation_handle", "<B1-Motion>", self.on_rotation_drag)
        self.canvas.tag_bind("rotation_handle", "<ButtonRelease-1>", self.on_rotation_release)
        
        # Add binding to handle clicks on the canvas background
        self.canvas.bind("<ButtonPress-1>", self.on_canvas_press)
    
    def on_canvas_press(self, event):
        """Handle clicks on the canvas background"""
        # Check if we clicked on an element or handle
        clicked_items = self.canvas.find_withtag("current")
        if not clicked_items:
            # Clicked on empty canvas, deselect all elements
            self.element_manager.deselect_all()
    
    def on_element_press(self, event):
        """Handle mouse press on an element"""
        if self._isResizing or self._isRotating:
            return
            
        clicked_items = self.canvas.find_withtag("current")
        if not clicked_items:
            return
            
        # Find which element was clicked
        for element_id, element in self.element_manager.elements.items():
            if element["canvas_id"] == clicked_items[0]:
                # If element not already selected, select it
                if element_id not in self.element_manager.selected_elements:
                    self.element_manager.deselect_all()
                    self.element_manager.select_element(element_id)
                
                self._isDragging = True
                self.drag_element = element_id
                self.drag_start_x = event.x
                self.drag_start_y = event.y
                break
    
    def on_element_drag(self, event):
        """Handle dragging an element"""
        if not self._isDragging:
            return
            
        dx = event.x - self.drag_start_x
        dy = event.y - self.drag_start_y
        
        element = self.element_manager.elements[self.drag_element]
        
        # Move the element and all its selection elements
        self.canvas.move(element["canvas_id"], dx, dy)
        
        if "selection" in element:
            selection = element["selection"]
            self.canvas.move(selection["rectangle"], dx, dy)
            for handle in selection["handles"]:
                self.canvas.move(handle, dx, dy)
            self.canvas.move(selection["rotation_handle"], dx, dy)
            self.canvas.move(selection["rotation_line"], dx, dy)
        
        # Update element position
        element["x"] += dx
        element["y"] += dy
        
        self.drag_start_x = event.x
        self.drag_start_y = event.y
    
    def on_element_release(self, event):
        """Handle release after dragging an element"""
        self._isDragging = False
        self.drag_element = None
    
    def on_resize_press(self, event):
        """Handle mouse press on a resize handle"""
        self._isResizing = True
        clicked_handle = self.canvas.find_withtag("current")[0]
        
        # Find which handle and element this belongs to
        for element_id, element in self.element_manager.elements.items():
            if "selection" in element and clicked_handle in element["selection"]["handles"]:
                self.resize_element = element_id
                self.resize_handle = clicked_handle
                
                # Determine handle type (nw, n, ne, etc.) from the tags
                handle_tags = self.canvas.gettags(clicked_handle)
                for tag in handle_tags:
                    if "resize" in tag:
                        self.resize_direction = tag
                        break
                
                # Store original element coordinates for reference
                canvas_id = element["canvas_id"]
                self.resize_start_coords = self.canvas.bbox(canvas_id)
                break
    
    def on_resize_drag(self, event):
        """Handle dragging a resize handle"""
        if not self._isResizing:
            return
            
        element = self.element_manager.elements[self.resize_element]
        canvas_id = element["canvas_id"]
        
        # Original coordinates
        x1, y1, x2, y2 = self.resize_start_coords
        
        # New dimensions based on resize direction
        if "nw-resize" in self.resize_direction:
            nx1, ny1 = event.x, event.y
            nx2, ny2 = x2, y2
        elif "n-resize" in self.resize_direction:
            nx1, ny1 = x1, event.y
            nx2, ny2 = x2, y2
        elif "ne-resize" in self.resize_direction:
            nx1, ny1 = x1, event.y
            nx2, ny2 = event.x, y2
        elif "e-resize" in self.resize_direction:
            nx1, ny1 = x1, y1
            nx2, ny2 = event.x, y2
        elif "se-resize" in self.resize_direction:
            nx1, ny1 = x1, y1
            nx2, ny2 = event.x, event.y
        elif "s-resize" in self.resize_direction:
            nx1, ny1 = x1, y1
            nx2, ny2 = x2, event.y
        elif "sw-resize" in self.resize_direction:
            nx1, ny1 = event.x, y1
            nx2, ny2 = x2, event.y
        elif "w-resize" in self.resize_direction:
            nx1, ny1 = event.x, y1
            nx2, ny2 = x2, y2
        
        # Apply the resize based on element type
        if element["type"] == "text":
            # For text, we can only adjust the position
            dx = (nx1 - x1) if "w" in self.resize_direction else 0
            dy = (ny1 - y1) if "n" in self.resize_direction else 0
            
            self.canvas.move(canvas_id, dx, dy)
            element["x"] += dx
            element["y"] += dy
            
        elif element["type"] == "image":
            # For images, we can adjust both position and size
            width = nx2 - nx1
            height = ny2 - ny1
            
            if width <= 10 or height <= 10:
                return  # Don't allow tiny sizes
                
            # Update element properties
            element["width"] = width
            element["height"] = height
            
            # Recreate the image with new dimensions
            img = Image.open(element["image_path"])
            img = img.resize((int(width), int(height)), Image.LANCZOS)
            photo = ImageTk.PhotoImage(img)
            
            # Store new photo reference
            self.element_manager.photo_references[self.resize_element] = photo
            
            # Update position if needed
            if "n" in self.resize_direction or "w" in self.resize_direction:
                element["x"] = nx1
                element["y"] = ny1
            
            # Delete old image and create new one
            self.canvas.delete(canvas_id)
            new_id = self.canvas.create_image(
                element["x"], element["y"], 
                image=photo, anchor="nw",
                tags=(self.resize_element, "element", "image")
            )
            element["canvas_id"] = new_id
        
        # Remove and redraw selection handles
        self.element_manager.remove_highlight(self.resize_element)
        self.element_manager.highlight_element(self.resize_element)
    
    def on_resize_release(self, event):
        """Handle release after resizing"""
        self._isResizing = False
        self.resize_element = None
        self.resize_handle = None
    
    def on_rotation_press(self, event):
        """Handle mouse press on rotation handle"""
        self._isRotating = True
        clicked_handle = self.canvas.find_withtag("current")[0]
        
        # Find which element this rotation handle belongs to
        for element_id, element in self.element_manager.elements.items():
            if "selection" in element and element["selection"]["rotation_handle"] == clicked_handle:
                self.rotation_element = element_id
                
                # Calculate center of the element
                canvas_id = element["canvas_id"]
                bbox = self.canvas.bbox(canvas_id)
                self.rotation_center_x = (bbox[0] + bbox[2]) / 2
                self.rotation_center_y = (bbox[1] + bbox[3]) / 2
                
                # Calculate initial angle
                dx = event.x - self.rotation_center_x
                dy = event.y - self.rotation_center_y
                self.rotation_start_angle = np.degrees(np.arctan2(dy, dx))
                
                # Store current rotation
                self.current_rotation = element.get("rotation", 0)
                break
    
    def on_rotation_drag(self, event):
        """Handle dragging the rotation handle"""
        if not self._isRotating:
            return
            
        # Calculate new angle
        dx = event.x - self.rotation_center_x
        dy = event.y - self.rotation_center_y
        new_angle = np.degrees(np.arctan2(dy, dx))
        
        # Calculate rotation angle difference
        angle_diff = new_angle - self.rotation_start_angle
        new_rotation = (self.current_rotation + angle_diff) % 360
        
        element = self.element_manager.elements[self.rotation_element]
        canvas_id = element["canvas_id"]
        
        # Apply rotation
        self.canvas.delete(canvas_id)
        
        if element["type"] == "text":
            # Create new text with rotation
            new_id = self.canvas.create_text(
                element["x"], element["y"],
                text=element["text"],
                font=element["font"],
                fill=element["color"],
                angle=new_rotation,
                tags=(self.rotation_element, "element", "text")
            )
            
        elif element["type"] == "image":
            # For image, we need to actually rotate the image
            img = Image.open(element["image_path"])
            img = img.resize((int(element["width"]), int(element["height"])), Image.LANCZOS)
            img = img.rotate(-new_rotation, expand=True, resample=Image.BICUBIC)
            photo = ImageTk.PhotoImage(img)
            
            # Store new photo reference
            self.element_manager.photo_references[self.rotation_element] = photo
            
            new_id = self.canvas.create_image(
                element["x"], element["y"],
                image=photo, anchor="nw",
                tags=(self.rotation_element, "element", "image")
            )
        
        element["canvas_id"] = new_id
        element["rotation"] = new_rotation
        
        # Remove and redraw selection handles
        self.element_manager.remove_highlight(self.rotation_element)
        self.element_manager.highlight_element(self.rotation_element)
    
    def on_rotation_release(self, event):
        """Handle release after rotation"""
        self._isRotating = False
        self.rotation_element = None
    
    def set_opacity(self, element_id, opacity):
        """Set the opacity of an element"""
        if element_id in self.element_manager.elements:
            element = self.element_manager.elements[element_id]
            element["opacity"] = opacity
            
            # Apply opacity based on element type
            if element["type"] == "image":
                # For images, we need to recreate the image with new opacity
                img = Image.open(element["image_path"])
                img = img.resize((int(element["width"]), int(element["height"])), Image.LANCZOS)
                
                # If opacity less than 1, convert to RGBA and adjust alpha channel
                if opacity < 1.0:
                    if img.mode != 'RGBA':
                        img = img.convert('RGBA')
                    alpha = int(opacity * 255)
                    img.putalpha(alpha)
                
                photo = ImageTk.PhotoImage(img)
                
                # Store new photo reference
                self.element_manager.photo_references[element_id] = photo
                
                # Update canvas
                self.canvas.delete(element["canvas_id"])
                new_id = self.canvas.create_image(
                    element["x"], element["y"],
                    image=photo, anchor="nw",
                    tags=(element_id, "element", "image")
                )
                element["canvas_id"] = new_id
            
            elif element["type"] == "text":
                # For text, adjust color transparency
                color = element["color"]
                if color.startswith("#") and len(color) == 7:
                    # Convert to RGBA
                    r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
                    alpha = int(opacity * 255)
                    rgba = f"#{r:02x}{g:02x}{b:02x}{alpha:02x}"
                    
                    # Update canvas
                    self.canvas.itemconfig(element["canvas_id"], fill=rgba)
                    element["color_with_opacity"] = rgba
            
            # If element is selected, redraw selection
            if element_id in self.element_manager.selected_elements:
                self.element_manager.remove_highlight(element_id)
                self.element_manager.highlight_element(element_id)

# Grouping Operations
Implement multi-select, group, and ungroup functionalities for elements.